In [28]:
from datasets import load_dataset
from Alz_preprocessing import get_train_test_datasets
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [29]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

def compute_weights(hf_split):

    #Compute class weights for a Hugging Face dataset split.

    #Args: hf_split: A Hugging Face dataset split (e.g., dataset['train'])

    #Returns: class_weights (dict): A dictionary mapping class index to its weight.

    labels = np.array(hf_split['label'])
    classes = np.unique(labels)

    weights = compute_class_weight(class_weight='balanced', classes=classes, y=labels)
    return dict(zip(classes, weights))

if __name__ == "__main__":
    from datasets import load_dataset
    dataset = load_dataset("Falah/Alzheimer_MRI")
    print(compute_weights(dataset['train']))

{np.int64(0): np.float64(1.7679558011049723), np.int64(1): np.float64(26.122448979591837), np.int64(2): np.float64(0.49883086515978176), np.int64(3): np.float64(0.7186973610331274)}


In [31]:
dataset = load_dataset("Falah/Alzheimer_MRI")
label_names = dataset['train'].features['label'].names
train_ds, test_ds = get_train_test_datasets(dataset, label_names)
class_weights = compute_weights(dataset['train'])
print("Class Weights:", class_weights)

TypeError: hf_to_tf_dataset() takes from 1 to 2 positional arguments but 3 were given